# Projet 3 (LU3IN005) : Chaines de Markov et épidémiologie :
# Propagation d’une épidémie dans une population

### <u>Groupe 6 :</u>
<pr>Samy (Adem) YAZID<br>Numéro étudiant : $3811684$<br><br>Margaux MARSELOO<br>Numéro étudiant : $28630611$</pr> 

### <u>**Consignes :**</u>
<pr>

L’objectif de ce projet est de manipuler des chaînes de Markov pour étudier la propagation d’une épidémie dans une population.<br>
Votre rendu sera un notebook, il est attendu que les codes soient commentés et les résultats interprétés.<br>
Les packages <em>numpy</em>, <em>random</em> et <em>matplotlib</em> sont conseillés.<br>
Nous allons étudier des populations constituées de $3$ types d’individus.<br>
Chaque individu est dans un des $3$ états suivant : sain $S$, infecté $I$ ou guéri $R$.
</pr>

In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random as rd

def rec_extract_values(iterable, dico):
    """
    Paramètres:
    iterable : any
        Iterable (ou non) dont on veut extraire les valeurs dans un dictionnaire
    dico : dict
        Dictionnaire qui stockera les valeurs extraites
        
    Retourne:
    None

    Description:
    Fonction permettant d'extraire toutes les valeurs de iterable en les stockant
    dans dico, et qui compte toutes les fois où chaque valeur est apparue.
    (Dans cette fonction les strings sont considérés comme non-iterable, donc
    seront compté comme une valeur quelconque)
    
    Exemple:
    dico : {}
    rec_extract_values([1, 3, [3, 7]])
    dico : {1:1, 3:2, 7:1}
    """
    if type(iterable) in [type([]), type(np.matrix([])), type(np.array([]))]:
        for l in iterable:
            rec_extract_values(l, dico)
        return dico
    else:
        if iterable not in dico:
            dico[iterable] = 1
        else:
            dico[iterable] += 1
            
def rec_sum_values(iterable):
    """
    Paramètres:
    iterable : any iterable type of (int or float)
        Iterable dont on veut sommer les valeurs
        
    Retourne:
    somme : int or float
        Somme des valeurs de iterable
    
    Description:
    Fonction permettant de retourner la somme de toutes les valeurs de iterable.
    
    Exemple:
    rec_sum_values([1, 3, [3, 7.5]]) : 14.5
    """
    if type(iterable) in [type([]), type(np.matrix([])), type(np.array([]))]:
        somme = 0
        for l in iterable:
            somme += rec_sum_values(l)
        return somme
    else:
        return iterable
    
def dict_zero(dico):
    """
    Paramètres:
    dico : dict
        Dictionnaire dont on veut mettre toutes ses valeurs à 0
        
    Retourne:
    None
    
    Description:
    Fonction permettant pour un dictionnaire de conserver ses clés tout en fixant leur valeur à 0.
    
    Exemple:
    dico : {1:1, 3:2, 7:1}
    dict_zero(dico)
    dico : {1:0, 3:0, 7:0}
    """
    for key in dico:
        dico[key] = 0

## I) <u>Apprentissage des paramètres d’un modèle à partir de données</u>

<pr>
    
Dans notre modèle, nous allons considérer qu’à chaque temps :
- Chaque individu sain peut rester sain ou devenir infecté
- Chaque individu infecté peut rester infecté ou devenir guéri
- Chaque individu guéri reste guéri
et que la probabilité de passer d’un état à l’autre ne dépend que de l’état précédent.

Nous disposons d’une séquence d’observations et nous souhaitons apprendre les paramètres de la
chaîne de Markov permettant de modéliser le processus sous-jacent qui a généré la séquence.<br>
Nous avons suivi un individu pendant $10$ jours, afin de déterminer a chaque temps dans quel état se trouvait l’individu.<br>
Nous avons obtenu la séquence d’observation suivante : $S, S, S, I, I ,I ,I , I, I, R$.
</pr>


<pr><u>**Question 1:**</u> À partir de cette séquence d’observation, estimez de les probabilités de transition entre les états et dresser la matrice de probabilité de transitions.</pr>


Matrice de transition du modèle :
$
\begin{pmatrix}
  &   S &   I &   R \\
S & 2/3 & 1/3 &   0 \\
I &   0 & 5/6 & 1/6 \\
R &   0 &   0 &   1 \\
\end{pmatrix}
$

In [3]:
matrice_transition = np.array([[2/3, 1/3,   0], 
                               [  0, 5/6, 1/6], 
                               [  0,   0,   1]])

<pr>
Nous avons ensuite suivi une population de $5000$ individus, pendant $200$ jours.<br>
Pour lire les données, utiliser <code>np.loadtxt("data_exo_2022.txt")</code>.<br>
Les individus sains sont notés $0$, les infectés $1$ et les guéris $2$.

<u>**Question 2:**</u><br>
    
1) Lire des données
    
</pr>

In [4]:
data = np.loadtxt("data_exo_2022.txt")

<pr>
    
2) Estimez les probabilités de transition entre les états et dressez la matrice de probabilité de transitions
(indice pour vérifier vos résultats : la première ligne de la matrice est $[0.9308, 0.0691, 0.]$).

</pr>

In [5]:
def analyse_seq(sequence, opt_card={}):
    sequence = sequence.astype(int)
    
    if opt_card == {}:
        card = {elem : 0 for elem in set(sequence)}
        nb_etats = len(card)
        mat_transition_etats = np.zeros((nb_etats, nb_etats))
        
        #on détermine la fréquence
        for etat in range(1, len(sequence)):
            card[sequence[etat-1]] += 1
            mat_transition_etats[sequence[etat-1]][sequence[etat]] += 1
        card[sequence[etat]] += 1
        mat_transition_etats[sequence[etat]][sequence[etat]] += 1

    else:
        card = opt_card
        nb_etats = len(card)
        mat_transition_etats = np.zeros((nb_etats, nb_etats))
        
        #on détermine la fréquence
        for etat in range(1, len(sequence)):
            mat_transition_etats[sequence[etat-1]][sequence[etat]] += 1
        mat_transition_etats[sequence[etat]][sequence[etat]] += 1
        
    return mat_transition_etats, card
    
def matrice_proba_transition(sequence, opt_card={}):
    mat_transition_etats, card = analyse_seq(sequence, opt_card)
    #on détermine les probabilités
    nb_etats = len(mat_transition_etats)
    for i in range(nb_etats):
        for j in range (nb_etats):
            mat_transition_etats[i][j] /= card[i]
    return mat_transition_etats
    

def matrice_proba_transition_liste(liste_sequence):
    card = {}
    rec_extract_values(liste_sequence, card)
    
    matrice_finale = np.zeros((len(card), len(card)))
    
    for seq in liste_sequence:
        dict_zero(card)
        rec_extract_values(seq, card)
        matrice_finale += matrice_proba_transition(seq, card)
        
    for i in range(len(matrice_finale)):
        for j in range(len(matrice_finale[0])):
            matrice_finale[i][j] /= (len(liste_sequence))
            
    return matrice_finale

print("Matrice de transition de l'exemple:\n")
print(matrice_proba_transition_liste([np.array([0., 0., 0., 1., 1., 1., 1., 1., 1., 2.])]))
print("\nMatrice de transition des 5000 individus:\n")
print(matrice_proba_transition_liste(data))

Matrice de transition de l'exemple:

[[0.66666667 0.33333333 0.        ]
 [0.         0.83333333 0.16666667]
 [0.         0.         1.        ]]

Matrice de transition des 5000 individus:

[[0.91574267 0.08425733 0.        ]
 [0.         0.9205482  0.0794518 ]
 [0.04864806 0.         0.95135194]]


## II) <u>Description du premier modèle</u>
<pr>La probabilité pour un individu d’être dans un de ces $3$ états au temps $t$, ne dépend que l’état dans
lequel il est au temps $t − 1$.<br>
Un individu dans l’état sain a une probabilité de $0.92$ de rester sain et une probabilité de $0.08$ de
devenir infecté. Si l’individu est infecté, il peut le rester avec une probabilité de $0.93$ et être guéri avec
une probabilité de $0.07$. S’il est dans l’état guéri, il reste dans cet état avec une probabilité de $1$.

<u>**Question 1:**</u> À partir du graphe de transition, créez la matrice de transition $A$, la matrice contenant les probabilités de transition entre les différents états. Créez une fonction permettant de vérifier qu’une matrice est stochastique.
</pr>

Matrice de transition du premier modèle :<br>
$A = 
\begin{pmatrix}
  &    S &    I &    R \\
S & 0.92 & 0.08 &    0 \\
I &    0 & 0.93 & 0.07 \\
R &    0 &    0 &    1 \\
\end{pmatrix}
$

In [6]:
matrice_transition_modele1 = np.array([[0.92, 0.08,    0], 
                                       [   0, 0.93, 0.07], 
                                       [   0,    0,    1]])

def est_stochastique(matrice):
    epsilon = 0.00001
    for ligne in matrice:
        if abs(rec_sum_values(ligne) - 1) >= epsilon:
            return False
    return True

print("La matrice de transition de la partie I) est-elle stochastique ? :", est_stochastique(matrice_transition))

print("La matrice de transition de la partie II) est-elle stochastique ? :", est_stochastique(matrice_transition))

La matrice de transition de la partie I) est-elle stochastique ? : True
La matrice de transition de la partie II) est-elle stochastique ? : True


<pr>
Au temps $t = 0$, un individu a une probabilité de $0.9$ d’être sain et $0.1$ d’être infecté.

<u>**Question 2:**</u> Créez $π_0$ la distribution de probabilité initiale.
</pr>

$
π_0 = 
\begin{pmatrix}
   S &    I &    R \\
0.90 & 0.10 &    0 \\
\end{pmatrix}
$

In [7]:
pi_0 = [0.90,
        0.10,
           0]

<pr>
Nous allons étudier l’évolution du nombre d’individu sains, infectés et guéris au cours du temps.
Dans un premier temps nous étudierons la distribution théorique, puis la distribution observée sur des simulations.
</pr>

### <u>Distribution $π_t$<u>

<pr><u>**Question 1:**</u> En utilisant $π_0$ et $A$, donnez la probabilité pour un individu d’être sain, infecté ou guéri au temps $t = 1$ (faire d’abord le calcul à la main).
</pr>


$
π_1 = π_0 * A =
\begin{pmatrix}
   S &    I &    R \\
0.90*0.92 + 0*0 & 0.90*0.08 + 0.10*0.93 &    0.10*0.07 + 0*1 \\
\end{pmatrix}
=
\begin{pmatrix}
   S &    I &    R \\
0.828 & 0.165 & 0.007\\
\end{pmatrix}
$

In [8]:
pi_1 = [0.90*0.92 + 0*0      ,
        0.90*0.08 + 0.10*0.93,
        0.10*0.07 + 0*1      ]

print("pi_1 =", pi_1)

pi_1 = [0.8280000000000001, 0.16500000000000004, 0.007000000000000001]


<pr><u>**Question 2:**</u> Donnez la probabilité pour un individu d’être sain, infecté ou guéri au temps $t = 2$ (faire d’abord le calcul à la main).
</pr>

$
π_2 = π_1 * A =
\begin{pmatrix}
   S &    I &    R \\
0.828*0.92 + 0.007*0   &   0.828*0.08 + 0.165*0.93   &   0.165*0.07 + 0.007*1 \\
\end{pmatrix}
=
\begin{pmatrix}
   S &    I &    R \\
0.76176 & 0.21969 & 0.01855\\
\end{pmatrix}
$

In [9]:
pi_2 = [0.828*0.92 + 0.007*0   ,
        0.828*0.08 + 0.165*0.93,
        0.165*0.07 + 0.007*1   ]

print("pi_2 =", pi_2)

pi_2 = [0.76176, 0.21969, 0.01855]


<pr><u>**Question 3:**</u> De même pour chaque temps $t$ entre $1$ et $200$, calculez la distribution théorique des effectifs dans chaque état (<em>Rappel:</em> $π_{t+1} = π_t . A$ ).
</pr>

In [10]:
def liste_pi(pi_0, A, n):
    l_pi = [pi_0]
    for i in range(n):
        pi_i = []
        for j in range(len(pi_0)):
            pi_i.append(l_pi[-1][j] * A[j][j] + l_pi[-1][j-1] * A[j-1][j])
        l_pi.append(pi_i)
    return l_pi

liste_pi_i = np.array(liste_pi(pi_0, matrice_transition_modele1, 200))
print("Liste des pi_i pour i entre 0 et 200 :")
print(liste_pi_i)

Liste des pi_i pour i entre 0 et 200 :
[[9.00000000e-01 1.00000000e-01 0.00000000e+00]
 [8.28000000e-01 1.65000000e-01 7.00000000e-03]
 [7.61760000e-01 2.19690000e-01 1.85500000e-02]
 [7.00819200e-01 2.65252500e-01 3.39283000e-02]
 [6.44753664e-01 3.02750361e-01 5.24959750e-02]
 [5.93173371e-01 3.33138129e-01 7.36885003e-02]
 [5.45719501e-01 3.57272330e-01 9.70081693e-02]
 [5.02061941e-01 3.75920827e-01 1.22017232e-01]
 [4.61896986e-01 3.89771324e-01 1.48331690e-01]
 [4.24945227e-01 3.99439090e-01 1.75615683e-01]
 [3.90949609e-01 4.05473972e-01 2.03576419e-01]
 [3.59673640e-01 4.08366763e-01 2.31959597e-01]
 [3.30899749e-01 4.08554980e-01 2.60545271e-01]
 [3.04427769e-01 4.06428112e-01 2.89144119e-01]
 [2.80073547e-01 4.02332365e-01 3.17594087e-01]
 [2.57667664e-01 3.96574984e-01 3.45757353e-01]
 [2.37054251e-01 3.89428148e-01 3.73517602e-01]
 [2.18089911e-01 3.81132518e-01 4.00777572e-01]
 [2.00642718e-01 3.71900434e-01 4.27456848e-01]
 [1.84591300e-01 3.61918821e-01 4.53489879e-01]
 

<pr><u>**Question 4:**</u> Représentez graphiquement la probabilité d’être dans chaque état en fonction du temps. (+ décrivez un peu ce que vous observez).
</pr>

In [11]:
def affiche_graphe_multi(liste_pi):
    #on crée les ordonnées
    liste_sains = []
    liste_infectes = []
    liste_gueris = []
    for pi in liste_pi_i:
        liste_sains.append(pi[0])
        liste_infectes.append(pi[1])
        liste_gueris.append(pi[2])
    
    #on crée les abscisses
    abscisses = [i for i in range(len(liste_pi))]
    
    #on plot le graphe comme une courbe
    plt.plot(abscisses, liste_sains)
    plt.plot(abscisses, liste_infectes)
    plt.plot(abscisses, liste_gueris)
    
    #on légende le graphe
    plt.legend(["Personnes saines", "Personnes infectées", "Personnes guéries"])
    
    #on nomme l'axe des abscisses
    plt.xlabel("Temps t")
    #on nomme l'axe des ordonnées
    plt.ylabel("Proportion")
    
    #on paramètre l'échelle des abscisses suivant l'option x_scale
    plt.xscale("linear")
    #on paramètre l'échelle des ordonnées comme linéaire
    plt.yscale("linear")
    
    #on donne un titre au graphique
    plt.title("Graphique des proportions de population en fonction du temps")
       
    #on affiche le graphe
    plt.show()

#affiche_graphe_multi(liste_pi_i)

<img src="Graph_Partie2.png"  align="bottom">

### <u>Tirage aléatoire des états</u>
<pr>Vous allez générer une séquence de taille $T$ en utilisant cette chaîne de Markov. Pour générer une séquence aléatoire, choisissiez un état initial au hasard (en utilisant $π_0$) ; puis choisissez les états suivants en suivant les probabilités de transition <em>(= la matrice de transition $A$).</em>
Vous pouvez prendre $T=150$.
</pr>

In [12]:
def genere_sequence(pi_0, A, n):
    p = rd.random()
    sequence = []
    for i in range(len(pi_0)):
        if p < sum(pi_0[:i+1]):
            sequence.append(i)
            break
    for i in range(1, n):
        p = rd.random()
        for j in range(len(A)):   
            if p < sum(A[sequence[-1]][:j+1]):
                sequence.append(j)
                break
    return np.array(sequence)

print("La séquence aléatoire de taille 150 suivant cette chaîne de Markov est :\n")
print(genere_sequence(pi_0, matrice_transition_modele1, 150))

La séquence aléatoire de taille 150 suivant cette chaîne de Markov est :

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


### <u>Modélisation d’une population</u>
<pr>Vous avez généré une séquence d’état pour un individu. Maintenant vous allez générer un ensemble de séquences pour une population de 200 individus.

<u>**Question 1:**</u> À chaque temps $t$, comptez le nombre d’individus sains, infectés et guéris dans la population et affichez l’évolution du nombre d’individus dans les trois états en fonction du temps.
</pr>

In [13]:
def genere_liste_sequence(pi_0, A, nb_individu, temps):
    liste_sequence = []
    for i in range(nb_individu):
        liste_sequence.append(genere_sequence(pi_0, A, temps))
    return np.array(liste_sequence)

def comptage_liste_sequence(liste_sequence, nb_etats=3):
    liste_comptage = []
    for jour in range(len(liste_sequence[0])):
        comptage = [0 for i in range(nb_etats)]
        for seq in liste_sequence:
            comptage[seq[jour]] += 1
        liste_comptage.append(comptage)
    return np.array(liste_comptage)

liste_sequence = genere_liste_sequence(pi_0, matrice_transition_modele1, 200, 150)
liste_comptage = comptage_liste_sequence(liste_sequence)

print("La liste de 200 séquences est :\n")
print(liste_sequence)

print("\n\nLa liste de 200 séquences compte cet effectif au cours du temps:\n")
print(liste_comptage)

La liste de 200 séquences est :

[[0 0 0 ... 2 2 2]
 [0 0 0 ... 2 2 2]
 [0 1 1 ... 2 2 2]
 ...
 [0 0 0 ... 2 2 2]
 [0 0 0 ... 2 2 2]
 [0 0 0 ... 2 2 2]]


La liste de 200 séquences compte cet effectif au cours du temps:

[[186  14   0]
 [175  24   1]
 [155  41   4]
 [146  49   5]
 [135  58   7]
 [131  58  11]
 [119  65  16]
 [113  68  19]
 [105  73  22]
 [ 93  83  24]
 [ 84  86  30]
 [ 80  82  38]
 [ 75  80  45]
 [ 71  78  51]
 [ 69  75  56]
 [ 64  71  65]
 [ 62  69  69]
 [ 58  67  75]
 [ 54  68  78]
 [ 49  69  82]
 [ 45  68  87]
 [ 40  65  95]
 [ 35  64 101]
 [ 32  63 105]
 [ 31  57 112]
 [ 29  54 117]
 [ 26  55 119]
 [ 25  48 127]
 [ 23  49 128]
 [ 23  44 133]
 [ 21  44 135]
 [ 20  42 138]
 [ 16  43 141]
 [ 15  41 144]
 [ 13  40 147]
 [ 11  40 149]
 [ 11  39 150]
 [ 11  35 154]
 [ 11  35 154]
 [  9  35 156]
 [  9  33 158]
 [  9  31 160]
 [  9  27 164]
 [  9  26 165]
 [  8  27 165]
 [  8  25 167]
 [  8  24 168]
 [  6  24 170]
 [  6  23 171]
 [  5  21 174]
 [  5  17 178]
 [  5  16 179]

<pr><u>**Question 2:**</u> Affichez le pourcentage d’individus sains infectés et guéris en fonction du temps.
</pr>

In [14]:
def pourcentage_liste_sequence(liste_sequence):
    liste_pourcentage = [[((cpt/sum(l_cpt))*100) for cpt in l_cpt] for l_cpt in comptage_liste_sequence(liste_sequence)]
    return np.array(liste_pourcentage)

liste_pourcentage = pourcentage_liste_sequence(liste_sequence)
print("\n\nLa liste de 200 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:")
print(liste_pourcentage)



La liste de 200 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:
[[ 93.    7.    0. ]
 [ 87.5  12.    0.5]
 [ 77.5  20.5   2. ]
 [ 73.   24.5   2.5]
 [ 67.5  29.    3.5]
 [ 65.5  29.    5.5]
 [ 59.5  32.5   8. ]
 [ 56.5  34.    9.5]
 [ 52.5  36.5  11. ]
 [ 46.5  41.5  12. ]
 [ 42.   43.   15. ]
 [ 40.   41.   19. ]
 [ 37.5  40.   22.5]
 [ 35.5  39.   25.5]
 [ 34.5  37.5  28. ]
 [ 32.   35.5  32.5]
 [ 31.   34.5  34.5]
 [ 29.   33.5  37.5]
 [ 27.   34.   39. ]
 [ 24.5  34.5  41. ]
 [ 22.5  34.   43.5]
 [ 20.   32.5  47.5]
 [ 17.5  32.   50.5]
 [ 16.   31.5  52.5]
 [ 15.5  28.5  56. ]
 [ 14.5  27.   58.5]
 [ 13.   27.5  59.5]
 [ 12.5  24.   63.5]
 [ 11.5  24.5  64. ]
 [ 11.5  22.   66.5]
 [ 10.5  22.   67.5]
 [ 10.   21.   69. ]
 [  8.   21.5  70.5]
 [  7.5  20.5  72. ]
 [  6.5  20.   73.5]
 [  5.5  20.   74.5]
 [  5.5  19.5  75. ]
 [  5.5  17.5  77. ]
 [  5.5  17.5  77. ]
 [  4.5  17.5  78. ]
 [  4.5  16.5  79. ]
 [  4.5  15.5  80. ]
 [  4.5  13.5 

<pr><u>**Question 3:**</u> Quand $t$ est grand, quelle est la proportion d’individus sains, infectés et guéris ?
</pr>

<pr>
Lorsque $t$ est grand, la répartition de la population est la suivante :
$ 
\begin{pmatrix}
   S &    I &     R \\
 0\% &  0\% & 100\% \\
\end{pmatrix}
$
</pr>

<pr><u>**Question 4:**</u> Refaites les questions précédentes avec des populations de tailles différentes, $5$ individus et $5000$ individus par exemple.
</pr>

In [15]:
#Pour 5 individus
liste_sequence = genere_liste_sequence(pi_0, matrice_transition_modele1, 5, 150)
liste_comptage = comptage_liste_sequence(liste_sequence)

print("La liste de 5 séquences est :\n")
print(liste_sequence)

print("\n\nLa liste de 5 séquences compte cet effectif au cours du temps:\n")
print(liste_comptage)

La liste de 5 séquences est :

[[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2]
 [0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [16]:
liste_pourcentage = pourcentage_liste_sequence(liste_sequence)
print("\n\nLa liste de 5 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:")
print(liste_pourcentage)



La liste de 5 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:
[[ 80.  20.   0.]
 [ 80.  20.   0.]
 [ 80.   0.  20.]
 [ 80.   0.  20.]
 [ 60.  20.  20.]
 [ 60.  20.  20.]
 [ 60.  20.  20.]
 [ 60.  20.  20.]
 [ 60.  20.  20.]
 [ 60.  20.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  20.  40.]
 [ 40.  20.  40.]
 [ 40.  20.  40.]
 [ 40.  20.  40.]
 [ 40.  20.  40.]
 [ 40.  20.  40.]
 [ 40.  20.  40.]
 [ 40.  20.  40.]
 [ 40.  20.  40.]
 [ 40.  20.  40.]
 [ 20.  40.  40.]
 [ 20.  40.  40.]
 [ 20.  40.  40.]
 [ 20.  40.  40.]
 [ 20.  40.  40.]
 [ 20.  40.  40.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.  20.  60.

<pr>
Lorsque $t$ est grand, la répartition de la population est, comme précédemment, la suivante :
$ 
\begin{pmatrix}
   S &    I &     R \\
 0\% &  0\% & 100\% \\
\end{pmatrix}
$
</pr>

In [17]:
#Pour 5000 individus
liste_sequence = genere_liste_sequence(pi_0, matrice_transition_modele1, 5000, 150)
liste_comptage = comptage_liste_sequence(liste_sequence)

print("La liste de 5000 séquences est :\n")
print(liste_sequence)

print("\n\nLa liste de 5000 séquences compte cet effectif au cours du temps:\n")
print(liste_comptage)

La liste de 5000 séquences est :

[[0 0 0 ... 2 2 2]
 [1 1 1 ... 2 2 2]
 [0 0 0 ... 2 2 2]
 ...
 [0 1 1 ... 2 2 2]
 [0 0 0 ... 2 2 2]
 [0 1 1 ... 2 2 2]]


La liste de 5000 séquences compte cet effectif au cours du temps:

[[4492  508    0]
 [4111  853   36]
 [3766 1142   92]
 [3485 1355  160]
 [3213 1530  257]
 [2935 1680  385]
 [2692 1788  520]
 [2483 1891  626]
 [2307 1933  760]
 [2118 1978  904]
 [1965 1992 1043]
 [1798 2016 1186]
 [1661 2008 1331]
 [1529 1999 1472]
 [1417 1959 1624]
 [1310 1949 1741]
 [1196 1935 1869]
 [1103 1893 2004]
 [1025 1837 2138]
 [ 935 1795 2270]
 [ 855 1741 2404]
 [ 790 1706 2504]
 [ 727 1642 2631]
 [ 664 1593 2743]
 [ 602 1543 2855]
 [ 555 1484 2961]
 [ 514 1425 3061]
 [ 486 1360 3154]
 [ 448 1313 3239]
 [ 412 1246 3342]
 [ 379 1199 3422]
 [ 346 1161 3493]
 [ 312 1116 3572]
 [ 289 1057 3654]
 [ 264  994 3742]
 [ 250  941 3809]
 [ 234  890 3876]
 [ 205  852 3943]
 [ 180  824 3996]
 [ 164  774 4062]
 [ 150  731 4119]
 [ 142  691 4167]
 [ 131  649 4220]
 [ 

In [18]:
liste_pourcentage = pourcentage_liste_sequence(liste_sequence)
print("\n\nLa liste de 5000 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:")
print(liste_pourcentage)



La liste de 5000 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:
[[8.984e+01 1.016e+01 0.000e+00]
 [8.222e+01 1.706e+01 7.200e-01]
 [7.532e+01 2.284e+01 1.840e+00]
 [6.970e+01 2.710e+01 3.200e+00]
 [6.426e+01 3.060e+01 5.140e+00]
 [5.870e+01 3.360e+01 7.700e+00]
 [5.384e+01 3.576e+01 1.040e+01]
 [4.966e+01 3.782e+01 1.252e+01]
 [4.614e+01 3.866e+01 1.520e+01]
 [4.236e+01 3.956e+01 1.808e+01]
 [3.930e+01 3.984e+01 2.086e+01]
 [3.596e+01 4.032e+01 2.372e+01]
 [3.322e+01 4.016e+01 2.662e+01]
 [3.058e+01 3.998e+01 2.944e+01]
 [2.834e+01 3.918e+01 3.248e+01]
 [2.620e+01 3.898e+01 3.482e+01]
 [2.392e+01 3.870e+01 3.738e+01]
 [2.206e+01 3.786e+01 4.008e+01]
 [2.050e+01 3.674e+01 4.276e+01]
 [1.870e+01 3.590e+01 4.540e+01]
 [1.710e+01 3.482e+01 4.808e+01]
 [1.580e+01 3.412e+01 5.008e+01]
 [1.454e+01 3.284e+01 5.262e+01]
 [1.328e+01 3.186e+01 5.486e+01]
 [1.204e+01 3.086e+01 5.710e+01]
 [1.110e+01 2.968e+01 5.922e+01]
 [1.028e+01 2.850e+01 6.122e+01]
 [9.

<pr>
Lorsque $t$ est grand, la répartition de la population est, encore une fois, la suivante :
$ 
\begin{pmatrix}
   S &    I &     R \\
 0\% &  0\% & 100\% \\
\end{pmatrix}
$
</pr>

### <u>Longueur de l’infection</u>
<pr>

<u>**Question 1:**</u> À partir des simulations, estimez la longueur moyenne d’une séquence de $I$.
</pr>

In [19]:
def longueur_moyenne(liste_sequence, categorie):
    moy = 0
    for seq in liste_sequence:
        for etat in seq:
            if etat == categorie:
                moy += 1
    return moy/len(liste_sequence)

longueur_moyenne_infectes = longueur_moyenne(liste_sequence, 1)

print("La longueur moyenne d'une séquence de I est de", longueur_moyenne_infectes, "jours")

La longueur moyenne d'une séquence de I est de 14.2764 jours


<u>**Question 2:**</u> Afficher la distribution observée de la longueur d’infection (peut-être que pour représenter une distribution, ça serait une bonne idée de faire un histogramme).
</pr>

In [27]:
def repartition_longueur_moyenne(liste_sequence, categorie):
    dico_repart = {}
    for seq in liste_sequence:
        cpt = 0
        for etat in seq:
            if etat == categorie:
                cpt += 1
        if cpt in dico_repart:
            dico_repart[cpt] += 1
        else:
            dico_repart[cpt] = 1
    return dico_repart

def affiche_graphe_repartition(dico_repart):

    #on crée les ordonnées
    ordonnees = [key for key in dico_repart]
    ordonnees.sort()
    
    #on crée les abscisses
    abscisses = [dico_repart[key] for key in ordonnees]
    
    #on plot l'histogramme
    plt.hist(abscisses, ordonnees)
    
    #on légende le graphe
    #plt.legend(["Personnes saines", "Personnes infectées", "Personnes guéries"])
    
    #on nomme l'axe des abscisses
    plt.xlabel("Longueur de l'inféction (en jours)")
    #on nomme l'axe des ordonnées
    plt.ylabel("Nombre de séquences")
    
    #on paramètre l'échelle des abscisses suivant l'option x_scale
    plt.xscale("linear")
    #on paramètre l'échelle des ordonnées comme linéaire
    plt.yscale("linear")
    
    #on donne un titre au graphique
    plt.title("Histogramme de la répartition de la longueur de l'infection")
       
    #on affiche le graphe
    plt.show()

dico_repart = repartition_longueur_moyenne(liste_sequence, 1)
print(dico_repart)

affiche_graphe_repartition(dico_repart)

{25: 61, 3: 292, 2: 319, 1: 359, 9: 195, 18: 89, 23: 79, 30: 40, 19: 104, 22: 77, 10: 167, 12: 154, 6: 240, 11: 185, 7: 221, 8: 197, 17: 103, 4: 289, 21: 103, 13: 137, 16: 116, 38: 28, 29: 63, 50: 13, 41: 17, 71: 8, 53: 18, 5: 274, 35: 34, 14: 132, 36: 34, 33: 26, 15: 127, 48: 10, 46: 15, 67: 3, 24: 65, 42: 14, 26: 60, 52: 12, 37: 26, 27: 52, 20: 91, 32: 40, 28: 56, 39: 14, 58: 7, 34: 29, 84: 3, 47: 15, 44: 11, 40: 15, 61: 4, 31: 38, 82: 3, 91: 1, 43: 14, 57: 7, 51: 5, 54: 5, 59: 8, 60: 7, 49: 15, 62: 5, 78: 1, 55: 12, 63: 5, 88: 2, 65: 4, 45: 11, 77: 2, 85: 1, 56: 3, 73: 1, 89: 1, 69: 2, 75: 2, 93: 1, 90: 1}


<u>**Question 3:**</u> Calculez la longueur théorique d’une séquence de $I$ (vous pourrez utiliser l’espérance de la loi géométrique en justifiant).
</pr>

<u>**Question 4:**</u> Comparer la longueur estimée et la longueur théorique.
</pr>

<u>**Question 5:**</u> Affichez la distribution théorique de la longueur d’infection.
</pr>

### <u>Petites modifications autour de ce premier modèle</u>
<pr>Vous pouvez maintenant modifier le modèle pour étudier différents cas de figure, en
faisant varier certains paramètres :
</pr>

<u>**Question 1:**</u> Faire varier la taille de la population.
</pr>

<u>**Question 2:**</u> Faire varier la distribution de probabilité initiale.
</pr>

<u>**Question 3:**</u> Faire varier les probabilités de transition.
</pr>

## III) <u>Description du second modèle</u>
<pr>Nous allons maintenant considérer un second modèle, les individus guéris peuvent redevenir sains avec une probabilité de $0.02$. Ils-elles peuvent perdre leur immunité face à la maladie.
</pr>

### <u>Analyse du modèle</u>

<pr>

<u>**Question 1:**</u> Ce processus peut-il être modélisé par une chaîne de Markov ?
</pr>

<pr>

<u>**Question 2:**</u>
Donnez la nouvelle matrice de transition.
</pr>

Matrice de transition du second modèle :<br>
$A = 
\begin{pmatrix}
  &    S &    I &    R \\
S & 0.92 & 0.08 &    0 \\
I &    0 & 0.93 & 0.07 \\
R & 0.02 &    0 & 0.98 \\
\end{pmatrix}
$

In [20]:
mat_transition_modele2 = np.array([[0.92, 0.08,    0], 
                                   [   0, 0.93, 0.07], 
                                   [0.02,    0, 0.98]])

<pr>
    
<u>**Question 3:**</u> Quels est la nature des états de cette chaîne de Markov ? Est-elle périodique ? Est-elle irréductible ?
</pr>

<pr>
    
<u>**Question 4:**</u> Calculez la matrice $A × A$. À quoi correspond-elle ? Est elle stochastique ? Même question pour $A^3$ et $A^4$.
</pr>

<pr>
    
<u>**Question 5:**</u> Réalisez les nouvelles simulations. Comment la population évolue-t-elle ?
</pr>

<pr>
    
<u>**Question 6:**</u> Refaites les simulations avec une autre distribution de probabilité initiale (par exemple si au temps $t = 0$, nous avons $90%$ d’infectés et $10%$ de sains). Explorez d’autres initialisations et comme pour chaque question commentez vos observations.
</pr>

<pr>
    
<u>**Question 7:**</u> Calculez la distribution de probabilité stationnaire et comparez ce résultat avec les simulations pour
$t$ assez grand.
</pr>

### <u>Longueur de l’immunité</u>

<pr>
    
On peut se demander combien de temps un individu qui a été malade, reste protégé contre la maladie.
    
<u>**Question 1:**</u> À partir des simulations, estimez la longueur moyenne d’une séquence de $R$.
</pr>

<pr>
    
<u>**Question 2:**</u> Calculez la longueur théorique d’une séquence de $R$.
</pr>

<pr>
    
<u>**Question 3:**</u> Affichez la distribution théorique et la distribution observée de la longueur de l’immunité.
</pr>

### <u>Modifier le modèle</u>

<pr>
   
<u>**Question 1:**</u> Comment l’épidémie évolue-t-elle si vous modifiez la probabilité pour un individu sain de devenir infecté ? Quelle est la nouvelle distribution a l’équilibre ?
</pr>

<pr>
    
<u>**Question 2:**</u> Calculez la longueur théorique d’une séquence de $R$.
</pr>

## IV) <u>Confinement</u>
<pr>On peut imaginer que si des mesures de distanciation sociale sont mises en place, la probabilité de devenir infecté devient nulle.

Nous allons alterner entre les périodes de non distanciation et de distanciation :
- En période de non-confinement, nous utilisons la matrice de transition de l’exercice précédent
- En période de confinement, la probabilité de devenir infecté pour un individu sain devient nulle

<u>**Question 1:**</u> Commencez les simulations avec la matrice de transition de l’exercice précédent.
- On peut considérer qu’au temps initial tous les individus sont sains.
- Quand il y a $25%$ d’individus infectés dans la population, nous passons en période de distanciation. Le nombre d’individus infectés va décroître.
- Quand il y a moins de $10%$ d’infectés, le confinement est levé.
</pr>

<pr>
    
<u>**Question 2:**</u> Faites les simulations pour une population assez grande, représentez l’évolution du nombre d’individus à chaque temps $t$ (vous devriez voir des “vagues”), et notez les temps de confinement et de deconfinement.
</pr>

<pr>
    
<u>**Question 3:**</u> Combien de confinements / déconfinements sont nécessaires ?
</pr>

## V) <u>Discussion</u>

<pr>

<u>**Question 1:**</u> Quelles remarques critiques pouvez faire sur les modèles utilisés ?
</pr>

<pr>
    
<u>**Question 2:**</u> <em>(Optionnelle)</em> Proposez des améliorations et implémentez-les si possible.
</pr>